## Importing Libraries

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import math as mt
import warnings
# import stockstats
warnings.filterwarnings('ignore')
# import pandas_ta as ta
import plotly.express as px
import plotly.graph_objects as go
import math as mt
import os

#### EMA

In [6]:
def EMA(data):
    
    data['ema_5'] = data['close'].ewm(span=5, adjust=False).mean()
    data['ema_13'] = data['close'].ewm(span=13, adjust=False).mean()
    data['ema_26'] = data['close'].ewm(span=26, adjust=False).mean()

## Strategy (with Efficient Risk Management)

In [7]:
class Backtesting():
    def __init__(self,data,price_movement,risk_free_rate=0.01,market_returns=None,beta=1):
        self.data = data
        self.close = data['close']
        self.ema5 = data['ema_5']
        self.ema13 = data['ema_13']
        self.ema26 = data['ema_26']
        self.volume = data['volume']
        self.volume_high_5days = data['volume_high5']
        self.low = data['low']
        self.high = data['high']
        
        self.capital = 1000000
        self.initial_capital = 1000000
        self.trailing_sl_percent = 1
        self.take_profit_percent = 1000
        self.stocks = 0
        self.price_movement = price_movement
        
        self.capital_withdrawn = 0
        
        self.drawdown = 0
        self.max_drawdown = 0
        self.global_maxima = 0
        
        self.won = 0
        self.loss = 0
        
        self.capital_invested = 0
        
        self.current = 'no position'
        self.exit = []
        self.entry = []
        self.portfolio_value = []
        
        self.returns = 0
        self.sharpe = 0
        self.treynor = 0
        self.calmar = 0
        self.jensens_alpha = 0
        self.risk_free_rate = risk_free_rate
        self.market_returns = market_returns
        self.beta = beta
        self.annualized_return = 0
        
    def calculate_drawdown(self, i):
        self.global_maxima = max(self.global_maxima, self.portfolio_value[i])
        self.drawdown = (self.global_maxima - self.portfolio_value[i]) * 100 / self.global_maxima
        self.max_drawdown = max(self.max_drawdown, self.drawdown)
        
    def check_win_loss(self):
        if self.portfolio_value[self.exit[-1]] > self.portfolio_value[self.entry[-1]]:
            self.won += 1
        else:
            self.loss += 1
            
    def check_exit_condition(self,i):
        if self.current == 'full position':
            
            ## STOP LOSS
            if i <= self.entry[-1] + 5 and self.close[i] < self.low[self.entry[-1]-1]:
                
                self.current = 'no position'
                self.stocks = 0
                self.capital = self.portfolio_value[-1]
                self.exit.append(i)
                self.check_win_loss()
            elif i>self.entry[-1] + 5 and self.ema5[i] < self.ema26[i] and self.ema5[i-1]>self.ema26[i-1] and self.ema13[i] > self.ema26[i]:
                self.current = 'no position'
                self.capital = self.portfolio_value[-1]
                self.stocks = 0
                self.exit.append(i)
                self.check_win_loss()
            elif i > self.entry[-1] + 5 and self.ema5[i] < self.ema13[i] and self.ema5[i-1]>self.ema13[i-1]:
                
                self.current = 'half position stop loss'
                self.capital += (self.stocks - int(self.stocks/2))*self.close[i]
                self.stocks = int(self.stocks/2)
            
            ## BOOK PROFIT Target 1
            elif self.close[i] - self.close[i-1] >= self.price_movement:
                
                self.current = "half position book profit"
                self.capital += (self.stocks - int(self.stocks/2))*self.close[i]
                self.stocks = int(self.stocks/2)
                
            
        elif self.current == 'half position stop loss':
            
            if self.ema5[i] < self.ema26[i] and self.ema5[i-1]>self.ema26[i-1] and self.ema13[i] < self.ema26[i]:
                
                self.current = 'no position'
                self.capital = self.portfolio_value[-1]
                self.stocks = 0
                self.exit.append(i)
                self.check_win_loss()
                
            elif self.ema5[i] > self.ema13[i] and self.ema5[i-1] < self.ema13[i-1]:
                
                self.current = "full position"
                self.stocks += int(self.capital/self.close[i])
                self.capital -= self.close[i]*int(self.capital/self.close[i])
        
        ## BOOK PROFIT TARGET 2       
        elif self.current == "half position book profit":
            
            if self.ema5[i] < self.ema13[i] and self.ema5[i-1]>self.ema13[i-1]:
                
                self.current = "third fourth book profit"
                self.capital += (self.stocks - int(self.stocks/2))*self.close[i]
                self.capital_withdrawn = (self.stocks - int(self.stocks/2))*self.close[i]
                self.stocks = int(self.stocks/2)
            elif self.ema5[i] < self.ema26[i] and self.ema5[i-1]>self.ema26[i-1] and self.ema13[i] < self.ema26[i]:
                
                self.stocks = 0
                self.capital = self.portfolio_value[-1]
                self.current = "no position"
                self.exit.append(i)
                self.check_win_loss()
                
        elif self.current == "third fourth book profit":
            
            if self.ema5[i] > self.ema13[i]:
                
                self.stocks += int(self.capital_withdrawn/self.close[i])   
                self.capital -= (int(self.capital_withdrawn/self.close[i]))*self.close[i]
                self.current = 'half position book profit'
        # target 3 ----------------------------------
            elif self.ema5[i] < self.ema26[i] and self.ema5[i-1]>self.ema26[i-1] and self.ema13[i] < self.ema26[i]:
                
                self.stocks = 0
                self.capital = self.portfolio_value[-1]
                self.current = "no position"
                self.exit.append(i)
                self.check_win_loss()
    
    def calculate_portfolio_value(self, i):
        capital = self.capital + self.stocks * self.close[i]
        self.portfolio_value.append(capital)
        self.check_exit_condition(i)
        
    def enter_long_position(self,i):
        
        if self.ema13[i] > self.ema26[i] and self.ema5[i] > self.ema13[i] and self.volume[i] == self.volume_high_5days[i]:
            self.stocks = int(self.capital/self.close[i])
            self.portfolio_value.append(self.capital)
            self.maximum_capital = self.capital
            self.minimum_capital = self.capital
            self.capital = self.capital - self.close[i]*self.stocks
            self.current = "full position"
            self.entry.append(i)
        else:
            self.calculate_portfolio_value(i)
            
    def backtest(self):
        for i in range(len(self.data)-1):
            if self.current == 'no position':
                self.enter_long_position(i)
            else:
                self.calculate_portfolio_value(i)
            self.calculate_drawdown(i)
            
        if self.current != "no position":
            self.capital += self.stocks * self.close[len(self.data)-1]
            self.portfolio_value.append(self.capital)
            self.stocks = 0
            self.exit.append(len(self.data)-1)
            self.check_win_loss()
        else:
            self.portfolio_value.append(self.capital)
        self.calculate_drawdown(i)
        
        self.calculate_annualized_returns(len(self.data))
        self.calculate_sharpe_ratio()
        self.calculate_treynor_ratio()
        self.calculate_calmar_ratio()
        self.calculate_jensens_alpha()
        self.returns = (self.capital - self.initial_capital) * 100 / self.initial_capital
    
    def calculate_annualized_returns(self,periods):
        total_return=(self.capital-self.initial_capital)/self.initial_capital
        self.annualized_return=(((1+total_return)**(6552/periods))-1)*100
        
    def calculate_sharpe_ratio(self):
        returns=np.diff(self.portfolio_value)/self.portfolio_value[:-1]
        excess_returns=returns-(self.risk_free_rate/6552)
        self.sharpe=np.sqrt(6552)*np.mean(excess_returns)/np.std(excess_returns)
        
    def calculate_beta(self,market_returns):
        returns=(np.diff(self.portfolio_value)/self.portfolio_value[:-1])*100
        covariance_matrix=np.cov(returns,market_returns)
        covariance=covariance_matrix[0, 1]
        market_variance=np.var(market_returns)
        self.beta=covariance/market_variance
        
    def calculate_treynor_ratio(self):
        if self.market_returns is not None:
            total_return=(self.capital-self.initial_capital)/self.initial_capital
            self.treynor=(total_return-self.risk_free_rate)/self.beta
    
    def calculate_calmar_ratio(self):
        if self.max_drawdown!=0:
            self.calmar=self.annualized_return/(self.max_drawdown)
    
    def calculate_jensens_alpha(self):
        if self.market_returns is not None:
            total_return=(self.capital-self.initial_capital)/self.initial_capital
            self.jensens_alpha=(total_return-self.risk_free_rate)-(self.beta*(np.mean(self.market_returns)-self.risk_free_rate))
            

## Data

In [8]:
directory_path = 'C:/Users/ASUS/OneDrive/Desktop/FAC_Projects/Carlson_Capital_Projects/Carlson_Capital_EMA_Strategy/NIFTY100'
csv_file_names = pd.read_csv('NIFTY100_TICKERS.csv')
csv_file_names = csv_file_names['ticker']
risk_free_rate=0.02
market_returns=pd.read_csv("market_returns.csv")
beta=1  
returns = []
ticker = []
max_drawdown = []
win_by_loss = []
annualized_returns = []
sharpe_ratios = []
calmar_ratios = []
jensens_alphas = []
treynor_ratio = []


for file_name in csv_file_names:
    full_file_name = file_name + '_with_indicators_.csv'
    full_file_path = os.path.join(directory_path, full_file_name)
        
    df = pd.read_csv(full_file_path)
    df['timestamp'] = pd.to_datetime(df['date'])
    
    df['timestamp'] = df['timestamp'].dt.tz_localize(None)
    df = df[df['timestamp'].dt.minute % 15 == 0]
    df = df[df['timestamp'].dt.year > 2019]
    df['volume_high5'] = df['volume'].rolling(window=5).max()
    EMA(df)
    check = df.copy()
    check['price movement'] = check['close'] - check['close'].shift(1)
    check['crossover'] = (
        (check['ema_5'] > check['ema_26']) &
        (check['ema_13'] > check['ema_26']) &
        (check['ema_13'].shift(1) < check['ema_26'].shift(1))
    )
    check = check[check['crossover'] == True]
    df.reset_index(inplace=True)
    
    lol = Backtesting(df, check['price movement'].mean(), risk_free_rate=risk_free_rate, market_returns=market_returns, beta=beta)
    lol.backtest()
    
    returns.append(lol.returns)
    max_drawdown.append(lol.max_drawdown)
    ticker.append(file_name)
    win_by_loss.append(lol.won / lol.loss if lol.loss != 0 else float('inf'))
    annualized_returns.append(lol.annualized_return)
    sharpe_ratios.append(lol.sharpe)
    calmar_ratios.append(lol.calmar)
    jensens_alphas.append(lol.jensens_alpha)
    treynor_ratio.append(lol.treynor)

# Create a DataFrame with the results
Results = pd.DataFrame({
    'Returns': returns,
    'Maximum Drawdown': max_drawdown,
    'Win/Loss ratio': win_by_loss,
    'Annualized Returns': annualized_returns,
    'Sharpe Ratio': sharpe_ratios,
    'Calmar Ratio': calmar_ratios,
    'Jensen\'s Alpha': jensens_alphas,
    'Treynor Ratio': treynor_ratio
})
Results.index = ticker
Results = Results.sort_values(by='Returns', ascending=False)

## Results (Important Metrics)

In [9]:
Results.head(10)

,Returns,Maximum Drawdown,Win/Loss ratio,Annualized Returns,Sharpe Ratio,Calmar Ratio,Jensen's Alpha,Treynor Ratio
ADANIGREEN,289.401025,28.913337,0.800000,95.464142,2.470256,3.301734,2.890932,2.874010
TATAMOTORS,160.015425,26.832997,1.090909,60.169605,2.280085,2.242374,1.597076,1.580154
ADANIENT,141.520395,36.824621,1.166667,54.448213,1.759352,1.478582,1.412126,1.395204
SAIL,125.710790,24.240890,0.687500,49.383315,1.749491,2.037191,1.254030,1.237108
JINDALSTEL,103.021360,21.166150,0.648649,41.777515,1.572001,1.973789,1.027136,1.010214
CHOLAFIN,95.741560,25.454279,0.589744,39.248197,1.485699,1.541910,0.954338,0.937416
SBIN,80.700070,19.927293,0.787879,33.869312,1.739949,1.699644,0.803923,0.787001
APOLLOHOSP,68.887535,16.185773,0.666667,29.481025,1.431627,1.821416,0.685797,0.668875
WIPRO,60.880340,10.725586,0.800000,26.415048,1.601971,2.462807,0.605725,0.588803
PEL,59.735845,14.836204,0.694444,25.970917,1.101825,1.750510,0.594280,0.577358


In [10]:
# Results.to_csv('Final_Results.csv', index=ticker)